In [23]:
import os
import numpy as np
from sklearn.datasets        import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import label_binarize

PIXEL_RANGE = range(0, 255 + 1)
IMAGE_RANGE = range(0, 9 + 1)

def process(mnist):
    # Process dataset to return features and labels for CNN
    def features():
        # Transform features to be float32 sets of 1x28x28 
        # tensors with normalized pixel values.
        return np.divide(
            mnist.data, PIXEL_RANGE[-1]
        ).astype(np.float32).reshape((-1, 1, 28, 28))
    def labels():
        # Transform labels to be float32 sets of 1x10
        # tensors that are one-hot encoded.
        return label_binarize(
            mnist.target, classes=IMAGE_RANGE
        ).astype(np.float32).reshape((-1, 10))
    return features(), labels()


# Download the MNIST dataset
mnist            = fetch_mldata('MNIST original', data_home='.')
features, labels = process(mnist)

# Split train and test data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)


In [24]:
# Import CNTK
import cntk as c

# Display CNTK version
' '.join([
    c.__name__.upper(),
    c.__version__,
    str(c.device.all_devices()[0])
])

'CNTK 2.0 CPU'

In [30]:
class ConvNet():
    IMAGE_SHAPE        = (1, 28, 28)
    IMAGE_CLASSES      = [n for n in IMAGE_RANGE]
    IMAGE_CLASS_COUNT  = len(IMAGE_CLASSES)
    LEARNING_RATE      = 0.001
    EPOCH_COUNT        = 20
    BATCH_SIZE         = 64
    DROP_RATE          = 0.5
    HIDDEN_LAYER_COUNT = 96
    
    def __init__(self):
        self._build_inputs()
        self._build_layers()
        self._build_stack()
        self._build_trainer()
    
    def evaluate(self, fn, feeds):
        for epoch in range(self.EPOCH_COUNT):
            batch = self.batch(epoch, feeds)
            fn(epoch, batch)
    
    def train(self, data):
        self.trainer.train_minibatch(data)
    
    def summarize(self):
        self.trainer.summarize_training_progress()
    
    def log_parameters(self):
        c.logging.log_number_of_parameters(self.stack)
        print()
    
    def checkpoint(self, version):
        file_names = ['ConvNet', 'MNIST', '{}.dnn'.format(version)]
        self.stack.save(
            os.path.join('.', 'checkpoints', '_'.join(file_names))
        )
    
    def batch(self, epoch, data):
        def chunk(data):
            slice_begin = epoch * self.BATCH_SIZE
            slice_end   = slice_begin + self.BATCH_SIZE
            return data[slice_begin:slice_end]
        return {key: chunk(value) for key, value in data.items()}
    
    def _build_inputs(self):
        self.inputs = c.input_variable(self.IMAGE_SHAPE,       np.float32, name='inputs')
        self.labels = c.input_variable(self.IMAGE_CLASS_COUNT, np.float32, name='labels')
    
    def _build_layers(self):
        with c.layers.default_options(activation=c.ops.relu, pad=False):
            self.layers  = [
                c.layers.Convolution2D((5,5), 32, pad=True),
                c.layers.MaxPooling((3,3), (2,2)),
                c.layers.Convolution2D((3,3), 48),
                c.layers.MaxPooling((3,3), (2,2)),
                c.layers.Convolution2D((3,3), 64),
                c.layers.Dense(self.HIDDEN_LAYER_COUNT),
                c.layers.Dropout(self.DROP_RATE),
                c.layers.Dense(self.IMAGE_CLASS_COUNT, activation=None)
            ]
    
    def _build_stack(self):
        self.stack = self.inputs
        for layer in self.layers: self.stack = layer(self.stack)
        self.loss  = c.losses.cross_entropy_with_softmax(self.stack, self.labels)
        self.error = c.metrics.classification_error(self.stack, self.labels)
    
    def _build_trainer(self):
        schedule     = c.learning_rate_schedule(self.LEARNING_RATE, c.UnitType.minibatch)
        learner      = c.learners.sgd(self.stack.parameters, schedule)
        printer      = c.logging.ProgressPrinter(tag='Training', num_epochs=self.EPOCH_COUNT)
        self.trainer = c.Trainer(self.stack, (self.loss, self.error), learner, printer)


In [31]:
cnn = ConvNet()
def train(epoch, batch_map):
    cnn.train(batch_map)
    cnn.summarize()
cnn.log_parameters()
cnn.evaluate(train, {'inputs': train_features, 'labels': train_labels})

Training 98778 parameters in 10 parameter tensors.

Learning rate per minibatch: 0.001
Finished Epoch[1 of 20]: [Training] loss = 2.348565 * 64, metric = 89.06% * 64 0.357s (179.3 samples/s);
Finished Epoch[2 of 20]: [Training] loss = 2.321183 * 64, metric = 93.75% * 64 0.131s (488.5 samples/s);
Finished Epoch[3 of 20]: [Training] loss = 2.339155 * 64, metric = 95.31% * 64 0.363s (176.3 samples/s);
Finished Epoch[4 of 20]: [Training] loss = 2.346929 * 64, metric = 95.31% * 64 0.176s (363.6 samples/s);
Finished Epoch[5 of 20]: [Training] loss = 2.319661 * 64, metric = 85.94% * 64 0.144s (444.4 samples/s);
Finished Epoch[6 of 20]: [Training] loss = 2.321407 * 64, metric = 84.38% * 64 0.193s (331.6 samples/s);
Finished Epoch[7 of 20]: [Training] loss = 2.301657 * 64, metric = 89.06% * 64 0.132s (484.8 samples/s);
Finished Epoch[8 of 20]: [Training] loss = 2.306839 * 64, metric = 92.19% * 64 0.162s (395.1 samples/s);
Finished Epoch[9 of 20]: [Training] loss = 2.325986 * 64, metric = 87.50%